<a href="https://colab.research.google.com/github/Henry-0810/Artificial-Intelligence/blob/main/chinese_chess_search_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Search Tree Algorithm Project
## End Game Steps of Chinese Chess (Xiang Qi)
Since full-game Chinese Chess is too complex, I plan to only focus on specific endgame scenarios.
1. King + Chariot vs. King (Basic but useful)
2. King + Cannon vs. King + Soldier (Intermediate)
3. King + Horse vs. King (More complex, requires mobility evaluation)

These scenarios are chosen because:
- The search tree remains manageable.
- The AI can calculate winning or drawing strategies.
- It demonstrates Minimax’s effectiveness.

---

**Some extra information about chinese chess:**
1. Chariot only can move up, down, left and right. It can capture any chess pieces, acts like a Rook in classic chess.
2. Cannon only can move up, down, left and right. A cannon must jump over a chess piece in its path to capture opponent's chess piece.
3. Soldier can only move one step forward, but once it moves pass the river, which is the mid line of the chess board, it can then move one step left, right and forward.
4. The Horse moves one point horizontally or vertically, and then one point diagonally. It cannot move in a direction where there is a piece blocking it along the path of movement.

---

**References:**
- [XiangQi Guide](https://www.xiangqi.com/how-to-play-xiangqi)

### **Implementation**

In [5]:
class Current_Board():
  def __init__(self, size=[9,10]):
    self.board = [['.']*size[0]]*size[1]

  def display(self):
    b = self.board
    for i in range(len(b)):
      for j in range(len(b[i])):
        print(b[i, j])

In [6]:
board = Current_Board()
board.display

<bound method Current_Board.display of <__main__.Current_Board object at 0x7c34d3a62750>>